# 02 Mapping Assistant

Interactive tool for reviewing **unmapped ministries/categories** and assigning them to normalized categories.

### How it works:
1. Loads `data/unmapped_categories.csv` (newly discovered ministries)
2. Loads `data/mappings_cleaned.csv` (current mapping table)
3. Displays unmapped ministries and lets you assign normalized categories interactively
4. Saves results back to `mappings_cleaned.csv` **with backups**

---

In [25]:
from pathlib import Path
root = Path('/home/ampersand/iceland-spending-ai')
data_dir = root / 'data'
structured_dir = data_dir / 'structured_csvs'
normalized_dir = data_dir / 'normalized'
mappings_file = data_dir / 'mappings_cleaned.csv'
unmapped_output = data_dir / 'unmapped_after_propagation.csv'
unmapped_path = data_dir / 'unmapped_categories.csv'

import pandas as pd
from pathlib import Path
from ipywidgets import IntProgress, HTML, VBox
from IPython.display import display

# === PATH SETUP ===
root = Path('/home/ampersand/iceland-spending-ai')

normalized_dir.mkdir(parents=True, exist_ok=True)

if not mappings_file.exists():
    raise FileNotFoundError(f"❌ Missing mappings file: {mappings_file}")

# === LOAD MAPPINGS ===
mappings_df = pd.read_csv(mappings_file)
mappings_df['Ministry'] = mappings_df['Ministry'].astype(str).str.strip()
mappings_df['Category'] = mappings_df['Category'].astype(str).str.strip()
mapping_dict = dict(zip(mappings_df['Ministry'], mappings_df['Category']))

# === PROCESS STRUCTURED CSV FILES ===
csv_files = sorted([f for f in structured_dir.glob("*.csv")])
progress = IntProgress(min=0, max=len(csv_files), description="Processing:", bar_style='info')
status = HTML()
display(VBox([progress, status]))

missing_counts = {}

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    if "Ministry" not in df.columns:
        continue

    # Apply mappings
    df['normalized_category'] = df['Ministry'].map(mapping_dict)
    missing_mask = df['normalized_category'].isna()

    # Store count + actual ministries
    missing_ministries = sorted(df.loc[missing_mask, 'Ministry'].dropna().unique().tolist())
    missing_counts[csv_file.name] = {
        "count": missing_mask.sum(),
        "ministries": missing_ministries
    }

    # Fill missing with placeholder
    df['normalized_category'] = df['normalized_category'].fillna("Misc/Missing")

    # Save normalized file
    out_path = normalized_dir / csv_file.name.replace(".csv", "_normalized.csv")
    df.to_csv(out_path, index=False)

    # Update progress bar
    progress.value += 1
    status.value = f"<b>{csv_file.name}</b>: {len(df)} rows → {missing_mask.sum()} missing"

progress.bar_style = "success"
status.value = "<b>✅ All files processed.</b>"

# === REPORT MISSING MAPPINGS ===
missing_summary = {f: m for f, m in missing_counts.items() if m["count"] > 0}

if missing_summary:
    # Save CSV with counts + examples
    missing_df = pd.DataFrame([
        {
            "File": fname,
            "Missing_Count": info["count"],
            "Example_Unmapped": ", ".join(info["ministries"][:5])
        }
        for fname, info in missing_summary.items()
    ])
    missing_df.to_csv(unmapped_output, index=False)

    print("\n=== SUMMARY OF MISSING MAPPINGS ===")
    for fname, info in missing_summary.items():
        print(f"  • {fname}: {info['count']} missing")
        if info["ministries"]:
            print(f"      ↳ Examples: {', '.join(info['ministries'][:3])}")
    print(f"\n⚠️ Missing categories saved to: {unmapped_output}")
else:
    print("\n✅ All ministries mapped successfully. No unmapped categories remain.")


=== SUMMARY OF MISSING MAPPINGS ===
  • 2004rikisreikningurheild_parsed.csv: 38 missing
  • 2007Rikisreikn_Heildaryfirlit_parsed.csv: 7 missing
  • 2007Rikisreikn_arsreikningur_rikisadila_parsed.csv: 44 missing
  • 2012-RR-rikisadila_parsed.csv: 44 missing
  • RR-2005-heild_parsed.csv: 6 missing
  • RR-2008-R_parsed.csv: 6 missing
  • RR-2008-arsreikningurrikisadila_parsed.csv: 40 missing
  • RR2006_arsreikningur_rikisadila_parsed.csv: 43 missing
  • RRheild2000_parsed.csv: 35 missing
  • RRheild_2006-A_parsed.csv: 5 missing
  • Rikisr-05-S_parsed.csv: 30 missing
  • Rikisreikn-01_parsed.csv: 35 missing
  • Rikisreikn-02_parsed.csv: 34 missing
  • Rikisreikn-03_parsed.csv: 34 missing
  • Rikisreikn2009ArsreiknRikisadila_parsed.csv: 44 missing
  • Rikisreikn2009Heildaryfirlit_parsed.csv: 7 missing
  • Rikisreikningur-2011-arsreikningar-rikiisadila_parsed.csv: 42 missing
  • Rikisreikningur-2016-Arsreikningur-Rikisadila_parsed.csv: 38 missing
  • Rikisreikningur-2019_parsed.csv: 55 miss

In [26]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

# === Load mappings and categories ===
mappings_path = mappings_file
mappings_df = pd.read_csv(mappings_path)
mappings_df["Ministry"] = mappings_df["Ministry"].astype(str).str.strip()
mappings_df["Category"] = mappings_df["Category"].astype(str).str.strip()

all_categories = sorted(mappings_df["Category"].unique())

# === Widgets ===
file_dropdown = widgets.Dropdown(
    options=["-- Select CSV file --"] + list(missing_summary.keys()),
    description="File:",
    layout=widgets.Layout(width="600px")
)

ministry_dropdown = widgets.Dropdown(
    options=["-- Select ministry --"],
    description="Ministry:",
    layout=widgets.Layout(width="600px")
)

category_dropdown = widgets.Dropdown(
    options=["-- Select existing category --"] + all_categories,
    description="Category:",
    layout=widgets.Layout(width="600px")
)

new_category_box = widgets.Text(
    placeholder="Or type a NEW category",
    description="New Cat:",
    layout=widgets.Layout(width="600px")
)

save_button = widgets.Button(
    description="💾 Save Mapping & Update",
    button_style="success",
    layout=widgets.Layout(width="300px")
)

status_output = widgets.Output()

# === Step 1. Load unmapped ministries for a file ===
def on_file_select(change):
    ministry_dropdown.options = ["-- Select ministry --"]
    if change["new"] == "-- Select CSV file --":
        return

    file_path = structured_dir / change["new"]
    if not file_path.exists():
        with status_output:
            clear_output()
            print(f"⚠️ File not found: {file_path}")
        return

    df = pd.read_csv(file_path)
    if "Ministry" not in df.columns:
        with status_output:
            clear_output()
            print(f"⚠️ No Ministry column in {change['new']}")
        return

    unmapped = sorted(set(df["Ministry"].dropna()) - set(mappings_df["Ministry"]))
    ministry_dropdown.options = ["-- Select ministry --"] + unmapped

file_dropdown.observe(on_file_select, names="value")

# === Step 2. Save mapping ===
def on_save_clicked(_):
    selected_file = file_dropdown.value
    selected_ministry = ministry_dropdown.value

    if selected_file == "-- Select CSV file --" or selected_ministry == "-- Select ministry --":
        with status_output:
            clear_output()
            print("⚠️ Please select both a CSV and a ministry.")
        return

    # Determine category (either dropdown or new)
    category = new_category_box.value.strip() or (
        category_dropdown.value if category_dropdown.value != "-- Select existing category --" else None
    )

    if not category:
        with status_output:
            clear_output()
            print("⚠️ Please select or enter a category.")
        return

    # Update mappings DataFrame
    new_row = pd.DataFrame({"Ministry": [selected_ministry], "Category": [category]})
    mappings_df_updated = pd.concat([mappings_df, new_row], ignore_index=True)
    mappings_df_updated.drop_duplicates(subset="Ministry", keep="last", inplace=True)
    mappings_df_updated.to_csv(mappings_path, index=False)

    # Re-run normalization ONLY for this file
    df = pd.read_csv(structured_dir / selected_file)
    df["normalized_category"] = df["Ministry"].map(
        dict(zip(mappings_df_updated["Ministry"], mappings_df_updated["Category"]))
    )
    df["normalized_category"].fillna("Misc/Missing", inplace=True)
    out_path = normalized_dir / selected_file.replace(".csv", "_normalized.csv")
    df.to_csv(out_path, index=False)

    # Update available categories live
    category_dropdown.options = ["-- Select existing category --"] + sorted(
        mappings_df_updated["Category"].unique()
    )

    with status_output:
        clear_output()
        print(f"✅ Saved mapping: '{selected_ministry}' → '{category}'")
        print(f"🔄 Normalized file updated: {out_path}")

save_button.on_click(on_save_clicked)

# === Display UI ===
display(widgets.VBox([
    file_dropdown,
    ministry_dropdown,
    category_dropdown,
    new_category_box,
    save_button,
    status_output
]))

In [27]:
import pandas as pd
import os

from pathlib import Path

root = Path('/home/ampersand/iceland-spending-ai')
mappings_path = data_dir / "mappings_cleaned.csv"


if not unmapped_path.exists():
    raise FileNotFoundError(f"Missing file: {unmapped_path}")
if not mappings_path.exists():
    raise FileNotFoundError(f"Missing file: {mappings_path}")

unmapped_df = pd.read_csv(unmapped_path)
mappings_df = pd.read_csv(mappings_path)


print(f"📥 Loaded {len(unmapped_df)} unmapped entries.")
display(unmapped_df.head())

📥 Loaded 4283 unmapped entries.


,Unmapped_Ministry
0,"(Árið 2007: Félagsmálaráðuneyti, Skuldir og ei..."
1,"- 01 101 Forsætisráðuneyti, aðalskrifstofa -7.102"
2,"- 03 101 Utanríkisráðuneyti, aðalskrifstofa -3..."
3,"-44.100 — 40 101 Samgönguráðuneyti, aðalskrifs..."
4,0.2% frá árinu á undan. Gjöld ráðuneytisins námu


In [28]:
from ipywidgets import IntProgress, HTML, VBox

# Get existing categories and already-mapped ministries
existing_categories = sorted(mappings_df['Category'].dropna().unique())
already_mapped = set(mappings_df['Ministry'].dropna().str.strip())

# Ministries still missing normalized categories
missing_normalized = set()
for fname in sorted(os.listdir(structured_dir)):
    if not fname.endswith(".csv"):
        continue
    path = os.path.join(structured_dir, fname)
    try:
        df = pd.read_csv(path)
    except:
        continue
    # If normalized_category column exists, find missing
    if "normalized_category" in df.columns:
        missing = df[df["normalized_category"].isna()]
        if not missing.empty and "Ministry" in missing.columns:
            missing_normalized.update(
                missing["Ministry"].dropna().astype(str).str.strip().unique()
            )

# Combine unmapped + missing-normalized
combined_unmapped = set(unmapped_df['Unmapped_Ministry'].dropna().str.strip())
unmapped_list = sorted((combined_unmapped | missing_normalized) - already_mapped)

# Filter out garbage entries
filtered_unmapped_list = [
    m for m in unmapped_list
    if isinstance(m, str)
    and m.strip()
    and not m.strip().replace(".", "").replace("-", "").isdigit()
]

print(f"📌 Ministries requiring attention: {len(filtered_unmapped_list)}")

📌 Ministries requiring attention: 0


In [29]:
import difflib
import pandas as pd
from ipywidgets import interact, Dropdown, Button, VBox, HBox, HTML, IntProgress
from IPython.display import display
from datetime import datetime
import os

assignments = {}
total = len(filtered_unmapped_list)

# --- Progress bar UI ---
progress_bar = IntProgress(value=0, min=0, max=total, description="Progress:")
progress_label = HTML(f"<b>0 / {total} mapped (0%)</b>")
progress_box = VBox([progress_bar, progress_label])
display(progress_box)

def suggest_category(ministry):
    """Suggest the closest existing category for a given ministry."""
    matches = difflib.get_close_matches(ministry, existing_categories, n=1, cutoff=0.6)
    return matches[0] if matches else "Misc/Missing"

def update_progress():
    done = len(assignments)
    pct = (done / total) * 100 if total else 100
    progress_bar.value = done
    progress_label.value = f"<b>{done} / {total} mapped ({pct:.1f}%)</b>"

def map_next(index=0):
    if index >= total:
        print("🎉 All ministries mapped!")
        finalize_mappings()
        return
    
    ministry = filtered_unmapped_list[index]
    default_category = suggest_category(ministry)
    
    # Dropdown for manual override
    dropdown = Dropdown(
        options=existing_categories + ["Misc/Missing"],
        value=default_category,
        description="Category:"
    )
    
    confirm_btn = Button(description="Confirm", button_style="success")
    skip_btn = Button(description="Skip", button_style="warning")
    info = HTML(f"<b>Ministry:</b> {ministry}")
    
    def confirm(_):
        assignments[ministry] = dropdown.value
        update_progress()
        box.close()
        map_next(index + 1)
        
    def skip(_):
        assignments[ministry] = "Misc/Missing"
        update_progress()
        box.close()
        map_next(index + 1)
    
    confirm_btn.on_click(confirm)
    skip_btn.on_click(skip)
    
    box = VBox([info, dropdown, HBox([confirm_btn, skip_btn])])
    display(box)

def finalize_mappings():
    """Automatically append new mappings into mappings_cleaned.csv and save backup."""
    if not assignments:
        print("⚠️ No new mappings to save.")
        return
    
    
    # Save a backup before modifying
    if os.path.exists(mapping_path):
        os.rename(mapping_path, backup_path)
        print(f"🗄 Backup saved: {backup_path}")
    
    # Load existing mappings if available
    try:
        df_existing = pd.read_csv(backup_path)
    except FileNotFoundError:
        df_existing = pd.DataFrame(columns=["LineCode", "Ministry", "Category"])
    
    # Prepare new mappings DataFrame
    df_new = pd.DataFrame(
        [(None, m, c) for m, c in assignments.items()],
        columns=["LineCode", "Ministry", "Category"]
    )
    
    # Merge old + new and drop duplicates
    df_updated = pd.concat([df_existing, df_new], ignore_index=True)
    df_updated = df_updated.drop_duplicates(subset=["Ministry"], keep="last")
    
    # Save updated mapping file
    df_updated.to_csv(mapping_path, index=False)
    print(f"✅ Updated mappings saved to: {mapping_path}")
    print(f"➕ Added {len(df_new)} new mappings.")

# Start mapping process
map_next(0)

🎉 All ministries mapped!
⚠️ No new mappings to save.


In [30]:
import time
import shutil
from ipywidgets import Button

# Ensure backups directory exists
backup_dir = os.path.join(os.path.dirname(mappings_path), "backups")
os.makedirs(backup_dir, exist_ok=True)

def save_assignments(_):
    global mappings_df
    if not assignments:
        print("⚠️ No new assignments to save.")
        return

    # Add new mappings to dataframe
    new_rows = pd.DataFrame({
        "LineCode": [None] * len(assignments),
        "Ministry": list(assignments.keys()),
        "Category": list(assignments.values())
    })
    mappings_df = pd.concat([mappings_df, new_rows], ignore_index=True)

    # Backup old mappings_cleaned.csv
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    backup_path = os.path.join(backup_dir, f"mappings_cleaned_backup_{timestamp}.csv")
    shutil.copy(mappings_path, backup_path)

    # Save updated mappings_cleaned.csv
    mappings_df.to_csv(mappings_path, index=False)

    print(f"💾 Saved {len(assignments)} new mappings.")
    print(f"📂 Updated: {mappings_path}")
    print(f"📦 Backup created: {backup_path}")

# Add a save button
save_button = Button(description="💾 Save & Backup", button_style="success")
save_button.on_click(save_assignments)
display(save_button)

Button(button_style='success', description='💾 Save & Backup', style=ButtonStyle())

In [31]:
print(unmapped_df.shape)
print(unmapped_df.head(10))

(4283, 1)
                                   Unmapped_Ministry
0  (Árið 2007: Félagsmálaráðuneyti, Skuldir og ei...
1  - 01 101 Forsætisráðuneyti, aðalskrifstofa -7.102
2  - 03 101 Utanríkisráðuneyti, aðalskrifstofa -3...
3  -44.100 — 40 101 Samgönguráðuneyti, aðalskrifs...
4   0.2% frá árinu á undan. Gjöld ráðuneytisins námu
5  00 201 Alþingi 43.500 02 101 Menntamálaráðuney...
6  00 401 Hæstiréttu 7600 09101 Fjármálaráðuneyti...
7  00 Æðsta stjórn ríkisins 18.000 02 Mennta- og ...
8  00213 Aldarafmæli sjálfstæðis og fullveldis Ís...
9  00213 Aldarafmæli sjálfstæðis ráðuneyti, aðals...


In [32]:
import os
import pandas as pd


# Scan normalized CSVs for missing normalized categories
missing_counts = {}
for file in os.listdir(normalized_dir):
    if not file.endswith(".csv"):
        continue
    df = pd.read_csv(os.path.join(normalized_dir, file))
    if "normalized_category" not in df.columns:
        print(f"⚠️ {file} has no normalized_category column!")
        continue
    missing = df["normalized_category"].isna().sum()
    if missing > 0:
        missing_counts[file] = missing

if missing_counts:
    print("⚠️ Missing normalized categories found:")
    for f, count in missing_counts.items():
        print(f"  • {f}: {count} missing")
else:
    print("✅ All normalized CSVs are fully mapped and consistent.")

⚠️ Missing normalized categories found:
  • Rikisreikn2009Heildaryfirlit_parsed_normalized.csv: 5746 missing
  • Rikisreikn2009ArsreiknRikisadila_parsed_normalized.csv: 17401 missing
  • Rikisreikningur-arid-2014-Arsreikningar-rikisadila_parsed_clean.csv: 16421 missing
  • 2007Rikisreikn_Heildaryfirlit_parsed_clean.csv: 5302 missing
  • RRheild2000_parsed_clean.csv: 2915 missing
  • Rikisreikn-03_parsed_clean.csv: 4995 missing
  • rikisreikningur2012_parsed_normalized.csv: 5563 missing
  • 2004rikisreikningurheild_clean.csv: 26 missing
  • Rikisreikningur-arid-2015-Arsreikningar-rikisadila_parsed_clean.csv: 16791 missing
  • Rikisreikn2009ArsreiknRikisadila_parsed_clean.csv: 17401 missing
  • 2012-RR-rikisadila_parsed_normalized.csv: 16718 missing
  • all_years_parsed_clean.csv: 325780 missing
  • Rikisreikningur-arid-2015-Heildaryfirlit_parsed_normalized.csv: 6255 missing
  • Rikisreikningur_11102018_FJS_parsed_normalized.csv: 4022 missing
  • fb_210803_parsed_normalized.csv: 53 missi

In [15]:
import os
import pandas as pd

# === CONFIG ===

# === LOAD EXISTING MAPPINGS ===
mappings_df = pd.read_csv(mappings_file)
mapping_dict = dict(
    zip(
        mappings_df["Ministry"].str.strip(),
        mappings_df["Category"].str.strip()
    )
)

updated_files = []
unmapped_counts = {}
global_unmapped = set()

print("🔄 Propagating normalized categories into structured CSVs...")

# === ITERATE THROUGH STRUCTURED CSVs ===
for fname in sorted(os.listdir(structured_dir)):
    if not fname.endswith(".csv"):
        continue

    path = os.path.join(structured_dir, fname)
    df = pd.read_csv(path)

    # Skip CSVs missing a Ministry column
    if "Ministry" not in df.columns:
        continue

    # Map normalized categories
    df["normalized_category"] = df["Ministry"].str.strip().map(mapping_dict)

    # Find missing ministries for this file
    missing_mask = df["normalized_category"].isna()
    missing_count = missing_mask.sum()

    if missing_count > 0:
        unmapped_counts[fname] = missing_count
        global_unmapped.update(df.loc[missing_mask, "Ministry"].dropna().unique())

    # Save the updated CSV back to the structured folder
    df.to_csv(path, index=False)
    updated_files.append(fname)

# === EXPORT ALL GLOBAL UNMAPPED MINISTRIES ===
if global_unmapped:
    unmapped_df = pd.DataFrame(sorted(global_unmapped), columns=["Ministry"])
    unmapped_df.to_csv(unmapped_output, index=False)
    print(f"\n📝 Unmapped ministries exported to: {unmapped_output}")

    # === APPEND UNMAPPED MINISTRIES INTO MAPPINGS FILE ===
    existing_ministries = set(mappings_df["Ministry"].str.strip())
    new_entries = unmapped_df[~unmapped_df["Ministry"].str.strip().isin(existing_ministries)]
    if not new_entries.empty:
        new_entries = new_entries.assign(Category="Misc/Missing")
        mappings_df = pd.concat([mappings_df, new_entries], ignore_index=True)
        mappings_df.to_csv(mappings_file, index=False)
        print(f"➕ Added {len(new_entries)} new placeholder mappings to: {mappings_file}")

# === SUMMARY ===
print("\n=== PROPAGATION SUMMARY ===")
print(f"✅ Updated {len(updated_files)} structured CSVs")

if unmapped_counts:
    print("⚠️ Files still missing categories:")
    for fname, count in unmapped_counts.items():
        print(f"   • {fname}: {count} missing")
    print(f"\n📝 Full list of unmapped ministries saved to: {unmapped_output}")
    print("⚡ Next step: Update these new entries in mappings_cleaned.csv with real categories.")
else:
    print("🎉 All normalized categories successfully propagated!")

FileNotFoundError: [Errno 2] No such file or directory: 'data/mappings_cleaned.csv'

In [22]:
os.chdir("/home/ampersand/iceland-spending-ai")
print(os.getcwd())

/home/ampersand/iceland-spending-ai


In [23]:
import os

True


In [16]:
import subprocess
import json
import pandas as pd
from pathlib import Path

# === CONFIG ===
root = Path('/home/ampersand/iceland-spending-ai')
unmapped_file = data_dir / "unmapped_after_propagation.csv"

# === LOAD EXISTING MAPPINGS ===
mappings_df = pd.read_csv(mappings_file)
existing_categories = sorted(mappings_df["Category"].dropna().unique())

# === LOAD UNMAPPED MINISTRIES ===
if unmapped_file.exists():
    unmapped_df = pd.read_csv(unmapped_file)
    # Collect ministries from normalized outputs if available
    all_unmapped = unmapped_df.get("Unmapped_Ministry", unmapped_df.iloc[:, 0])
    unmapped_list = sorted(set(all_unmapped.dropna().str.strip()))
else:
    print("✅ No unmapped ministries found.")
    unmapped_list = []

# === CHECK FOR OLLAMA ===
def ollama_available():
    try:
        result = subprocess.run(["ollama", "--version"], capture_output=True)
        return result.returncode == 0
    except FileNotFoundError:
        return False

has_ollama = ollama_available()

# === FUNCTION TO CLASSIFY MINISTRIES ===
def classify_with_ollama(ministry, categories):
    prompt = f"""
    Given these categories: {', '.join(categories)},
    choose the single most appropriate category for the Icelandic ministry or institution below.
    Ministry/Institution: "{ministry}"
    Answer ONLY with the category name from the provided list.
    """
    result = subprocess.run(
        ["ollama", "run", "llama3", "--json"],
        input=prompt.encode(),
        capture_output=True
    )
    try:
        output = json.loads(result.stdout.decode())["response"].strip()
    except Exception:
        output = "Misc/Missing"
    return output

# === AUTO-SUGGEST MAPPINGS ===
suggested_mappings = {}

if has_ollama and unmapped_list:
    print(f"🤖 Using Ollama for {len(unmapped_list)} unmapped ministries...")
    for ministry in unmapped_list:
        suggested_category = classify_with_ollama(ministry, existing_categories)
        suggested_mappings[ministry] = suggested_category
    print("✅ Automatic classification completed.")
elif unmapped_list:
    print(f"⚠️ Ollama not found. {len(unmapped_list)} ministries remain unmapped.")
else:
    print("✅ Nothing to classify.")

# === UPDATE MAPPINGS FILE ===
if suggested_mappings:
    # Create a DataFrame of new mappings
    new_df = pd.DataFrame(
        list(suggested_mappings.items()), columns=["Ministry", "Category"]
    )
    # Append and drop duplicates just in case
    mappings_df = pd.concat([mappings_df, new_df], ignore_index=True)
    mappings_df.drop_duplicates(subset=["Ministry"], inplace=True)
    mappings_df.to_csv(mappings_file, index=False)
    print(f"💾 Updated mappings saved to: {mappings_file}")

🤖 Using Ollama for 36 unmapped ministries...
✅ Automatic classification completed.
💾 Updated mappings saved to: /home/ampersand/iceland-spending-ai/data/mappings_cleaned.csv
